In [3]:
import numpy as np
import pandas as pd
from typing import List, Optional

In [4]:
pd.set_option('display.max_rows', None)

In [5]:
def _output_size(I, K, P, S): 
    return int((I - K + 2*P)/(S)) + 1

activations_size = 6 
kernel_size = 4 
padding = 0
stride = 1
output_size = _output_size(activations_size, kernel_size, padding, stride)
num_patches = output_size ** 2

In [6]:
f"{output_size}x{output_size}"

'3x3'

In [7]:
I, J = np.indices((activations_size, activations_size))
activations = np.char.add(np.char.add('A', I.astype(str)), J.astype(str))

In [8]:
I, J = np.indices((kernel_size, kernel_size))
kernel = np.char.add(np.char.add('K', I.astype(str)), J.astype(str))

In [9]:
equations = []
patches = [] 
inv_patches = [] 

num_rows = (activations_size - kernel_size) // stride + 1
num_cols = (activations_size - kernel_size) // stride + 1

equations_matrix = [[None for _ in range(num_cols)] for _ in range(num_rows)]

for i in range(0, activations_size - kernel_size + 1, stride):
    for j in range(0, activations_size - kernel_size + 1, stride):
        patch = activations[i:i+kernel_size, j:j+kernel_size]
        inv_patch = activations[i:i+kernel_size, j:j+kernel_size].T
        patches.append(patch)
        inv_patches.append(inv_patch)
        
        terms = []
        for a in range(kernel_size):
            for b in range(kernel_size):
                terms.append(f"{patch[a, b]}*{kernel[a, b]}")
        equation = " + ".join(terms)
        
        row_idx = i // stride
        col_idx = j // stride
        equations_matrix[row_idx][col_idx] = equation

In [10]:
split_equations_dict = {'0': '000_0'}

for row_id in range(len(equations_matrix)):
    for col_id in range(len(equations_matrix[row_id])):
        equation = equations_matrix[row_id][col_id]
        terms = equation.split(" + ")
        split_eq = []

        for i in range(4):
            split_eq.append(" + ".join(terms[i*4 : (i+1)*4]))
        
        for i, eq in enumerate(split_eq):
            key = f"C{row_id}{col_id}_{i}"
            split_equations_dict[eq] = key

In [11]:
patches = np.array(patches)
inv_patches = np.array(inv_patches)

In [12]:
class PE:
    # links: [0: left, 1: up, 2: right, 3: down]
    def __init__(self, links: Optional[List['PE']] = None):
        if links is None:
            links = [None, None, None, None]
        self.links = links
        self.activation = "0"
        self.weight = '--'
        self.accumulation: str = "0"
    
    def _input(self, activation: str):
        self.activation = activation
    
    def _weight(self, weight: str):
        self.weight = weight
    
    def shift(self, shift_direction: int, activation_flag: bool):
        neighbor = self.links[shift_direction]
        if neighbor is not None:
            if activation_flag:
                neighbor._input(self.activation)
            else:
                neighbor._weight(self.weight)

class SystolicArray:
    def __init__(self, size: int):
        self.size = size
        self.buffer = []  
        self.array = self._setup_array()
    
    def _setup_array(self):
        array = [[PE() for _ in range(self.size)] for _ in range(self.size)]
        for i in range(self.size):
            for j in range(self.size):
                left = array[i][j-1] if j > 0 else None
                up = array[i-1][j] if i > 0 else None
                right = array[i][j+1] if j < self.size - 1 else None
                down = array[i+1][j] if i < self.size - 1 else None
                array[i][j].links = [left, up, right, down]
        return array

    def print_array(self):
        for i in range(self.size):
            row_parts = []
            for pe in self.array[i]:
                act_str = pe.activation.ljust(4)
                weight_str = pe.weight.ljust(4)
                
                if pe.accumulation.strip() == "0":
                    parts = ["0", "0", "0", "0"]
                else:
                    parts = [p.strip() for p in pe.accumulation.split('+')]
                    while len(parts) < 4:
                        parts.append("0")
                    parts = parts[:4]
                parts = [p.ljust(8) for p in parts]
                acc_str = " + ".join(parts)
                
                row_parts.append(f"[A:{act_str} | W:{weight_str} | Acc:{acc_str}]")
            print(" | ".join(row_parts))

    def cycle(self, new_activations: Optional[List[str]] = None):
        if new_activations:
            for i in range(self.size):
                #  shift activations rightward
                for j in range(self.size - 1, 0, -1):
                    self.array[i][j].activation = self.array[i][j-1].activation
                # shift the new activation into column 0.
                self.array[i][0].activation = new_activations[i]
        
        bottom_row = [self.array[self.size - 1][j].accumulation for j in range(self.size)]
        self.buffer.append(bottom_row)
        # Look to get the equations
        
        for j in range(self.size):
            # save previous cycle's accumulations for column j
            prev_acc = [self.array[i][j].accumulation for i in range(self.size)]
            
            # Update the accumulation in row 0 for column j.
            cell0 = self.array[0][j]
            prod0 = f"{cell0.activation}*{cell0.weight}" if cell0.activation not in ["0", "--"] else "0"
            cell0.accumulation = prod0
            
            for i in range(1, self.size):
                cell = self.array[i][j]
                prod = f"{cell.activation}*{cell.weight}" if cell.activation not in ["0", "--"] else "0"
                inherited = prev_acc[i-1]
                if inherited == "0":
                    new_acc = prod
                else:
                    new_acc = inherited + (f" + {prod}" if prod != "0" else "")
                cell.accumulation = new_acc if new_acc != "" else "0"

In [13]:
class Conv2dSimulator:
    def __init__(self, array_size: int):
        self.input_buffers = [
            [],              
            ["0"],           
            ["0", "0"],      
            ["0", "0", "0"] 
        ]
        
        self.systolic_array = SystolicArray(array_size)
        self.outputs = [] 

    def add_patches(self, patches: List[np.ndarray]):
        for i, patch in enumerate(patches):
            flat_patch = patch.flatten().tolist()
            # Use round-robin assignment across the buffers
            self.input_buffers[i % len(self.input_buffers)].extend(flat_patch)

    def cycle(self):
        new_activations = []
        for i in range(self.systolic_array.size):
            if self.input_buffers[i]:
                new_act = self.input_buffers[i].pop(0)
            else:
                new_act = "0"
            new_activations.append(new_act)
        
        print("New activations:", new_activations)
        self.systolic_array.cycle(new_activations=new_activations)
        self.systolic_array.print_array()
        output = self.systolic_array.buffer[-1]
        translated_equations = [] 
        for _out in output: 
            try: 
                translated_equations.append(split_equations_dict[_out])
            except:
                translated_equations.append('Cxx_x')
        self.outputs.append(translated_equations)
        print("Output: ", translated_equations)
        return translated_equations

In [14]:
conv_sim = Conv2dSimulator(kernel_size)
conv_sim.add_patches(patches)


kernel_test = kernel.T
for i in range(kernel_size):
    for j in range(kernel_size):
        conv_sim.systolic_array.array[i][j].weight = kernel_test[i, j]

for cycle_num in range(100):
    out = conv_sim.cycle()
    print()

New activations: ['A00', '0', '0', '0']
[A:A00  | W:K00  | Acc:A00*K00  + 0        + 0        + 0       ] | [A:0    | W:K10  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K20  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K30  | Acc:0        + 0        + 0        + 0       ]
[A:0    | W:K01  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K11  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K21  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K31  | Acc:0        + 0        + 0        + 0       ]
[A:0    | W:K02  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K12  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K22  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K32  | Acc:0        + 0        + 0        + 0       ]
[A:0    | W:K03  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K13  | Acc:0        + 0        + 0        + 0       ] | [A:0    | W:K2

In [15]:
pd.DataFrame(conv_sim.outputs)

,0,1,2,3
0,000_0,000_0,000_0,000_0
1,000_0,000_0,000_0,000_0
2,000_0,000_0,000_0,000_0
3,000_0,000_0,000_0,000_0
4,Cxx_x,000_0,000_0,000_0
5,Cxx_x,Cxx_x,000_0,000_0
6,Cxx_x,Cxx_x,Cxx_x,000_0
7,Cxx_x,Cxx_x,Cxx_x,Cxx_x
8,Cxx_x,Cxx_x,Cxx_x,Cxx_x
9,Cxx_x,Cxx_x,Cxx_x,Cxx_x


In [ ]:
for a in equations_matrix: 
    for b in a: 
        print(b)

['A00*K00 + A01*K01 + A02*K02 + A03*K03 + A10*K10 + A11*K11 + A12*K12 + A13*K13 + A20*K20 + A21*K21 + A22*K22 + A23*K23 + A30*K30 + A31*K31 + A32*K32 + A33*K33',
 'A01*K00 + A02*K01 + A03*K02 + A04*K03 + A11*K10 + A12*K11 + A13*K12 + A14*K13 + A21*K20 + A22*K21 + A23*K22 + A24*K23 + A31*K30 + A32*K31 + A33*K32 + A34*K33',
 'A02*K00 + A03*K01 + A04*K02 + A05*K03 + A12*K10 + A13*K11 + A14*K12 + A15*K13 + A22*K20 + A23*K21 + A24*K22 + A25*K23 + A32*K30 + A33*K31 + A34*K32 + A35*K33',
 'A10*K00 + A11*K01 + A12*K02 + A13*K03 + A20*K10 + A21*K11 + A22*K12 + A23*K13 + A30*K20 + A31*K21 + A32*K22 + A33*K23 + A40*K30 + A41*K31 + A42*K32 + A43*K33',
 'A11*K00 + A12*K01 + A13*K02 + A14*K03 + A21*K10 + A22*K11 + A23*K12 + A24*K13 + A31*K20 + A32*K21 + A33*K22 + A34*K23 + A41*K30 + A42*K31 + A43*K32 + A44*K33',
 'A12*K00 + A13*K01 + A14*K02 + A15*K03 + A22*K10 + A23*K11 + A24*K12 + A25*K13 + A32*K20 + A33*K21 + A34*K22 + A35*K23 + A42*K30 + A43*K31 + A44*K32 + A45*K33',
 'A20*K00 + A21*K01 + A22*K0

In [17]:
patches

array([[['A00', 'A01', 'A02', 'A03'],
        ['A10', 'A11', 'A12', 'A13'],
        ['A20', 'A21', 'A22', 'A23'],
        ['A30', 'A31', 'A32', 'A33']],

       [['A01', 'A02', 'A03', 'A04'],
        ['A11', 'A12', 'A13', 'A14'],
        ['A21', 'A22', 'A23', 'A24'],
        ['A31', 'A32', 'A33', 'A34']],

       [['A02', 'A03', 'A04', 'A05'],
        ['A12', 'A13', 'A14', 'A15'],
        ['A22', 'A23', 'A24', 'A25'],
        ['A32', 'A33', 'A34', 'A35']],

       [['A10', 'A11', 'A12', 'A13'],
        ['A20', 'A21', 'A22', 'A23'],
        ['A30', 'A31', 'A32', 'A33'],
        ['A40', 'A41', 'A42', 'A43']],

       [['A11', 'A12', 'A13', 'A14'],
        ['A21', 'A22', 'A23', 'A24'],
        ['A31', 'A32', 'A33', 'A34'],
        ['A41', 'A42', 'A43', 'A44']],

       [['A12', 'A13', 'A14', 'A15'],
        ['A22', 'A23', 'A24', 'A25'],
        ['A32', 'A33', 'A34', 'A35'],
        ['A42', 'A43', 'A44', 'A45']],

       [['A20', 'A21', 'A22', 'A23'],
        ['A30', 'A31', 'A32', 'A33'],
